In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np

# ------------------- Thuật toán mã hóa -------------------

# Caesar Cipher
def caesar_encrypt(plaintext, key):
    result = ""
    for char in plaintext:
        if char.isalpha():
            shift = ord('A') if char.isupper() else ord('a')
            result += chr((ord(char) - shift + key) % 26 + shift)
        else:
            result += char
    return result

def caesar_decrypt(ciphertext, key):
    return caesar_encrypt(ciphertext, -key)

# Vigenère Cipher
def vigenere_encrypt(plaintext, key):
    key = key.lower()
    key_repeated = ''.join([key[i % len(key)] for i in range(len(plaintext))])
    ciphertext = ""
    for p, k in zip(plaintext, key_repeated):
        if p.isalpha():
            shift = ord('A') if p.isupper() else ord('a')
            k_val = ord(k) - ord('a')
            ciphertext += chr((ord(p) - shift + k_val) % 26 + shift)
        else:
            ciphertext += p
    return ciphertext

def vigenere_decrypt(ciphertext, key):
    key = key.lower()
    key_repeated = ''.join([key[i % len(key)] for i in range(len(ciphertext))])
    plaintext = ""
    for c, k in zip(ciphertext, key_repeated):
        if c.isalpha():
            shift = ord('A') if c.isupper() else ord('a')
            k_val = ord(k) - ord('a')
            plaintext += chr((ord(c) - shift - k_val) % 26 + shift)
        else:
            plaintext += c
    return plaintext

# Playfair Cipher
def playfair_generate_key_matrix(key):
    matrix = []
    used = set()
    key = key.upper().replace("J", "I")
    for char in key:
        if char not in used and char.isalpha():
            used.add(char)
            matrix.append(char)
    for i in range(65, 91):
        if chr(i) == 'J': continue
        if chr(i) not in used:
            matrix.append(chr(i))
    return [matrix[i:i+5] for i in range(0, 25, 5)]

def playfair_find_position(matrix, char):
    for i in range(5):
        for j in range(5):
            if matrix[i][j] == char:
                return i, j
    return -1, -1

def playfair_process_text(text):
    text = text.upper().replace("J", "I")
    processed = ""
    i = 0
    while i < len(text):
        a = text[i]
        b = text[i+1] if i+1 < len(text) else 'X'
        if a == b:
            processed += a + 'X'
            i += 1
        else:
            processed += a + b
            i += 2
    if len(processed) % 2 == 1:
        processed += 'X'
    return processed

def playfair_encrypt(text, key):
    matrix = playfair_generate_key_matrix(key)
    text = playfair_process_text(text)
    ciphertext = ""
    for i in range(0, len(text), 2):
        a, b = text[i], text[i+1]
        r1, c1 = playfair_find_position(matrix, a)
        r2, c2 = playfair_find_position(matrix, b)
        if r1 == r2:
            ciphertext += matrix[r1][(c1+1)%5] + matrix[r2][(c2+1)%5]
        elif c1 == c2:
            ciphertext += matrix[(r1+1)%5][c1] + matrix[(r2+1)%5][c2]
        else:
            ciphertext += matrix[r1][c2] + matrix[r2][c1]
    return ciphertext

def playfair_decrypt(text, key):
    matrix = playfair_generate_key_matrix(key)
    plaintext = ""
    for i in range(0, len(text), 2):
        a, b = text[i], text[i+1]
        r1, c1 = playfair_find_position(matrix, a)
        r2, c2 = playfair_find_position(matrix, b)
        if r1 == r2:
            plaintext += matrix[r1][(c1-1)%5] + matrix[r2][(c2-1)%5]
        elif c1 == c2:
            plaintext += matrix[(r1-1)%5][c1] + matrix[(r2-1)%5][c2]
        else:
            plaintext += matrix[r1][c2] + matrix[r2][c1]
    return plaintext

# Hill Cipher
def hill_encrypt(plaintext, key_matrix):
    plaintext = plaintext.replace(" ", "").upper()
    while len(plaintext) % 2 != 0:
        plaintext += "X"
    numbers = [ord(c) - ord('A') for c in plaintext]
    ciphertext = ""
    for i in range(0, len(numbers), 2):
        block = np.array(numbers[i:i+2])
        encrypted = key_matrix.dot(block) % 26
        ciphertext += ''.join(chr(e + ord('A')) for e in encrypted)
    return ciphertext

def hill_decrypt(ciphertext, key_matrix):
    det = int(np.round(np.linalg.det(key_matrix)))
    det_inv = None
    for i in range(1, 26):
        if (det * i) % 26 == 1:
            det_inv = i
            break
    if det_inv is None:
        raise ValueError("Matrix is not invertible mod 26")
    adjugate = np.round(det * np.linalg.inv(key_matrix)).astype(int) % 26
    inverse_matrix = (det_inv * adjugate) % 26
    numbers = [ord(c) - ord('A') for c in ciphertext]
    plaintext = ""
    for i in range(0, len(numbers), 2):
        block = np.array(numbers[i:i+2])
        decrypted = inverse_matrix.dot(block) % 26
        plaintext += ''.join(chr(e + ord('A')) for e in decrypted)
    return plaintext

# Rail Fence Cipher
def rail_fence_encrypt(text, rails):
    if rails == 1:
        return text
    fence = [''] * rails
    rail = 0
    direction = 1
    for char in text:
        fence[rail] += char
        rail += direction
        if rail == 0 or rail == rails - 1:
            direction *= -1
    return ''.join(fence)

def rail_fence_decrypt(cipher, rails):
    fence = [['\n' for _ in cipher] for _ in range(rails)]
    rail, direction = 0, 1
    for i in range(len(cipher)):
        fence[rail][i] = '*'
        rail += direction
        if rail == 0 or rail == rails - 1:
            direction *= -1

    index = 0
    for r in range(rails):
        for c in range(len(cipher)):
            if fence[r][c] == '*' and index < len(cipher):
                fence[r][c] = cipher[index]
                index += 1

    result = ''
    rail, direction = 0, 1
    for i in range(len(cipher)):
        result += fence[rail][i]
        rail += direction
        if rail == 0 or rail == rails - 1:
            direction *= -1
    return result

# ------------------- Giao diện Tkinter -------------------

def run_cipher():
    text = entry_input.get()
    key = entry_key.get()
    mode = mode_var.get()
    algo = algo_var.get()

    try:
        if algo == "Caesar":
            key_int = int(key)
            result = caesar_encrypt(text, key_int) if mode == "Encrypt" else caesar_decrypt(text, key_int)
        elif algo == "Vigenere":
            result = vigenere_encrypt(text, key) if mode == "Encrypt" else vigenere_decrypt(text, key)
        elif algo == "Playfair":
            result = playfair_encrypt(text, key) if mode == "Encrypt" else playfair_decrypt(text, key)
        elif algo == "Hill":
            key_matrix = np.array([[int(x) for x in key.split()]])
            key_matrix = key_matrix.reshape(2, 2)
            result = hill_encrypt(text, key_matrix) if mode == "Encrypt" else hill_decrypt(text, key_matrix)
        elif algo == "Rail Fence":
            key_int = int(key)
            result = rail_fence_encrypt(text, key_int) if mode == "Encrypt" else rail_fence_decrypt(text, key_int)
        else:
            result = "Chưa chọn thuật toán."
        output_text.delete(0, tk.END)
        output_text.insert(0, result)
    except Exception as e:
        messagebox.showerror("Lỗi", str(e))

# Giao diện
root = tk.Tk()
root.title("Mã hóa & Giải mã")

tk.Label(root, text="Chọn thuật toán:").grid(row=0, column=0, sticky="w")
algo_var = tk.StringVar(value="Caesar")
for i, name in enumerate(["Caesar", "Vigenere", "Playfair", "Hill", "Rail Fence"]):
    tk.Radiobutton(root, text=name, variable=algo_var, value=name).grid(row=0, column=i+1)

tk.Label(root, text="Chế độ:").grid(row=1, column=0, sticky="w")
mode_var = tk.StringVar(value="Encrypt")
tk.Radiobutton(root, text="Mã hóa", variable=mode_var, value="Encrypt").grid(row=1, column=1)
tk.Radiobutton(root, text="Giải mã", variable=mode_var, value="Decrypt").grid(row=1, column=2)

tk.Label(root, text="Văn bản đầu vào:").grid(row=2, column=0, sticky="w")
entry_input = tk.Entry(root, width=60)
entry_input.grid(row=2, column=1, columnspan=4)

tk.Label(root, text="Khóa:").grid(row=3, column=0, sticky="w")
entry_key = tk.Entry(root, width=60)
entry_key.grid(row=3, column=1, columnspan=4)

tk.Label(root, text="Kết quả:").grid(row=4, column=0, sticky="w")
output_text = tk.Entry(root, width=60)
output_text.grid(row=4, column=1, columnspan=4)

tk.Button(root, text="Thực hiện", command=run_cipher).grid(row=5, column=0, columnspan=5)

root.mainloop()
